In [1]:
from tradinglab.data_manager import * 
from tradinglab.utils import * 
from datetime import datetime
from datetime import timedelta

In [2]:
stocks = data_manager(archive='FMIB_prices')
stocks.status()

First available record: 2019-01-02
Last available record:  2019-12-30
Total number of records: 362



**Find Volume explosion**

In [4]:
count = 0
stock_list = list()
stats = trading_chart()

for tick in stocks.ticker_list:
    price = stocks.stock_price(stock=tick, date_start='2019-06-01')
    # compute MAV
    stats.mav(price, var='Volume', window=20)
    # compute Volume-MAV ratio
    df = pd.concat([price['Volume'], stats.values], axis=1)
    df['ratio'] = df['Volume']/df['MAV20']
    # filter by ratio > Threshold
    exceed = df[df['ratio'] > 2.0]
    # set a time baseline to show events
    T0 = (datetime.now() - timedelta(days=20)).strftime("%Y-%m-%d")
    # print trading exceeding threshod in last N days
    if not exceed.loc[T0:].empty:
        count += 1
        print("{}) {}".format(count, stocks.stock_name))
        print(exceed.loc[T0:])
        print("---------------------------------------------\n")
        stock_list.append(stocks.stock_name)

1) ATL.MI
               Volume       MAV20     ratio
Date                                       
2019-12-20  6135834.0  2328126.75  2.635524
---------------------------------------------

2) CNHI.MI
               Volume       MAV20     ratio
Date                                       
2019-12-13  6839502.0  2994769.35  2.283816
---------------------------------------------

3) CPR.MI
               Volume       MAV20     ratio
Date                                       
2019-12-20  4460030.0  2039726.45  2.186582
---------------------------------------------

4) ENEL.MI
                Volume       MAV20     ratio
Date                                        
2019-12-20  77087165.0  29117023.2  2.647495
---------------------------------------------

5) EXO.MI
              Volume      MAV20     ratio
Date                                     
2019-12-20  744591.0  342319.35  2.175136
---------------------------------------------

6) FBK.MI
               Volume       MAV20     ratio
Da

**Find Bullish Engulfing**

In [ ]:
count = 0
stock_list = list()

def get_candle_type(pclose, popen):
    if pclose >= popen:
        return 'bullish'
    else:
        return 'bearish'

chart = trading_chart()
T0 = '2019-01-01'
for tick in stocks.ticker_list: # stocks.ticker_list
    price = stocks.stock_price(stock=tick, date_start=T0)
    
    # range of second candle is larger
    rng1 = price['Open']-price['Close']
    rng2 = price['Close']-price['Open']
    ratio = rng2/rng1.shift(1) # disabled
    
    # range of first candle is comparable with ATR to avoid candles too small 
    chart.ATR(price, window=14)
    ATR = chart.values
    isSized = (rng1 > 0.5 * ATR).shift(1)
    
    # oversold
    chart.stoch(price)
    oversold = chart.values[:,0] <= 20.
    
    # second candle fully enclose the first
    band = rng1.shift(1)/5.  # avoid equal border levels
    pattern1 = price['Close'].shift(1) - band > price['Open'] 
    pattern2 = price['Open'].shift(1) + band < price['Close'] 
    
    # first candle is bearish and second bullish 
    candle = price.apply(lambda x: get_candle_type(x['Close'], x['Open']), axis=1)
   
    ## Bullish Engulfing Pattern  
    engulf_bull = ( (candle.shift(1)=='bearish')&(candle=='bullish') & 
                   (pattern1 & pattern2) & isSized & oversold).values
    
    engulf_bull = ( (candle.shift(1)=='bearish')&(candle=='bullish') & 
                    (pattern1 & pattern2) & (ratio > 1.5) & isSized & oversold).values

    exceed = price.loc[engulf_bull]
    
    # set a time baseline
    #T0 = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    # print trading exceeding threshod in last N days
    if not exceed.loc[T0:].empty:
        count += 1
        print("{}) {}".format(count, stocks.stock_name))
        print(exceed.loc[T0:])
        print("------------------------------------------------------------------\n")
        stock_list.append(stocks.stock_name)

**Find Bearish Engulfing**

In [ ]:
count = 0
stock_list = list()

def get_candle_type(pclose, popen):
    if pclose >= popen:
        return 'bullish'
    else:
        return 'bearish'

chart = trading_chart()
T0 = '2019-01-01'
for tick in stocks.ticker_list:  # stocks.ticker_list
    price = stocks.stock_price(stock=tick, date_start=T0)
    
    # range of second candle is larger
    rng1 = price['Close']-price['Open']
    rng2 = price['Open']-price['Close']
    ratio = rng2/rng1.shift(1) # disabled
    
    # range of first candle is comparable with ATR to avoid candles too small 
    chart.ATR(price, window=14)
    ATR = chart.values
    isSized = (rng1 > 0.5 * ATR).shift(1)
    
    # overbought
    chart.stoch(price)
    overbought = chart.values[:,0] >= 80.
    
    # second candle fully enclose the first
    band = rng1.shift(1)/5.  # avoid equal border levels
    pattern1 = price['Open'].shift(1) - band > price['Close'] 
    pattern2 = price['Close'].shift(1) + band < price['Open'] 
    
    # first candle is bullish and second bearish
    candle = price.apply(lambda x: get_candle_type(x['Close'], x['Open']), axis=1)
   
    ## Bullish Engulfing Pattern
    engulf_bear = ( (candle.shift(1)=='bullish')&(candle=='bearish') & 
                    (pattern1 & pattern2) & isSized & overbought).values
    
    engulf_bear = ( (candle.shift(1)=='bullish')&(candle=='bearish') & 
                    (pattern1 & pattern2) & (ratio > 1.5) & isSized & overbought).values

    exceed = price.loc[engulf_bear]
    
    # set a time baseline
    #T0 = (datetime.now() - timedelta(days=20)).strftime("%Y-%m-%d")
    # print trading exceeding threshod in last N days
    if not exceed.loc[T0:].empty:
        count += 1
        print("{}) {}".format(count, stocks.stock_name))
        print(exceed.loc[T0:])
        print("------------------------------------------------------------------\n")
        stock_list.append(stocks.stock_name)

**Find Inversion Pattern**

In [ ]:
'''
hammer: pattern di inversione identificabile dopo un downtrend. Non necessita conferma dalle candele successive.
shooting-star: pattern di inversione collocato in un trend rialzista. Può essere anche pattern di continuazione del trend rialzista.
inverted-hammer: pattern di inversione rialzista identificabile dopo un downtrend. Necessita di conferma: apertura della candela successiva più alta del real body.
hanging-man: pattern di inversione ribassista identificabile dopo un uptrend prolungato.
https://www.ig.com/it/hammer
'''
pattern = 'hanging-man'

count = 0
stock_list = list()

chart = trading_chart()
T0 = '2019-01-01'
for tick in stocks.ticker_list: # stocks.ticker_list
    price = stocks.stock_price(stock=tick, date_start=T0)
    
    # range
    rng = np.abs(price['Open']-price['Close'])
    
    # shadow length
    lower_shadow = price[['Open', 'Close']].min(axis=1) - price['Low']
    upper_shadow = price['High'] - price[['Open', 'Close']].max(axis=1) 
    
    # shadow length comparable to ATR
    chart.ATR(price, window=14)
    ATR = chart.values
    
    if pattern == 'shooting-star':
        ls = (upper_shadow >= ATR)
        us = (lower_shadow <= 0.8 * rng)
        # oversold
        chart.stoch(price)
        oversold = chart.values[:,0] >= 80.
        
    if pattern == 'hammer':
        ls = (lower_shadow >= ATR)
        us = (upper_shadow <= 0.8 * rng)
        # oversold
        chart.stoch(price)
        oversold = chart.values[:,0] <= 20.
        
    if pattern == 'inverted-hammer':
        ls = (upper_shadow >= ATR)
        us = (lower_shadow <= 0.8 * rng)
        # oversold
        chart.stoch(price)
        oversold = chart.values[:,0] <= 20.
        
    if pattern == 'hanging-man':
        ls = (lower_shadow >= ATR)
        us = (upper_shadow <= 0.8 * rng)
        # oversold
        chart.stoch(price)
        oversold = chart.values[:,0] >= 80.
  
    ## Pattern  
    ix = ( ls & us & oversold).values

    exceed = price.loc[ix]
    
    # set a time baseline
    #T0 = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    # print trading exceeding threshod in last N days
    if not exceed.loc[T0:].empty:
        count += 1
        print("{}) {}".format(count, stocks.stock_name))
        print(exceed.loc[T0:])
        print("------------------------------------------------------------------\n")
        stock_list.append(stocks.stock_name)

In [ ]:
tick = 'PIRC.MI'
price = stocks.stock_price(stock=tick, date_start='2019-03-01')
chart = trading_chart(stock_name=tick, figsize=(16,8))
chart.candlestick(price, width=0.6)
chart.add_subplot()
chart.ATR(price, window=14)
chart.add_subplot()
chart.stoch(price)
#chart.volumes(price)
#chart.mav(price, var='Volume', window=20)
chart.show()

In [ ]:
tick = 'EXO.MI'
price = stocks.stock_price(stock=tick, date_start='2019-06-01')
chart = trading_chart(stock_name=tick, figsize=(16,8))
chart.candlestick(price, width=0.6)
chart.add_subplot()
chart.RSI(price, window=14)
chart.add_subplot()
chart.stoch(price)
#chart.volumes(price)
#chart.mav(price, var='Volume', window=20)
chart.show()

In [5]:
stocks.tickers()

Tickers: 
A2A.MI
AMP.MI
ATL.MI
AZM.MI
BAMI.MI
BPE.MI
BZU.MI
CNHI.MI
CPR.MI
DIA.MI
ENEL.MI
ENI.MI
EXO.MI
FBK.MI
FCA.MI
G.MI
HER.MI
IG.MI
ISP.MI
JUVE.MI
LDO.MI
MB.MI
MONC.MI
NEXI.MI
PIRC.MI
PRY.MI
PST.MI
RACE.MI
REC.MI
SFER.MI
SPM.MI
SRG.MI
STM.MI
TEN.MI
TIT.MI
TRN.MI
UBI.MI
UCG.MI
UNI.MI
US.MI
#40 tickers found in "data\FMIB_prices.pkl".

